In [1]:
!pip install ftfy

     |████████████████████████████████| 71kB 5.6MB/s 
  Created wheel for ftfy: filename=ftfy-6.0.3-cp37-none-any.whl size=41935 sha256=82e61964e40cfb35b0a5d1e7b2c86c7166b36cc0f1806d9140b113016186d30f
  Stored in directory: /root/.cache/pip/wheels/99/2c/e6/109c8a28fef7a443f67ba58df21fe1d0067ac3322e75e6b0b7
Successfully built ftfy


In [2]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [3]:
import re #matching regular expression
import ftfy #fixes text for you:fixes all the junk characters
import nltk#natural language toolkit
import itertools #for iterations to matrix
import numpy as np #for array
import warnings #overcome warnings
warnings.filterwarnings("ignore")
import pandas as pd #read data
import pickle as pkl #store tokenizer
from pathlib import Path #create json file
from nltk import PorterStemmer #for stemming process
import matplotlib.pyplot as plt #plot confusion matrix
from nltk.corpus import stopwords #for remove stop words
from sklearn.model_selection import train_test_split #split data
from keras.models import Model, Sequential #adding layers one by one
from keras.optimizers import Adam #for optimize the training
from keras.preprocessing.text import Tokenizer #to genarate unique number for uniq words
from keras.preprocessing.sequence import pad_sequences #for making all the record has same shape
from sklearn.metrics import  classification_report, confusion_matrix, accuracy_score #for evaluate the model accuracy
from keras.layers import Conv1D, Dense, Input, LSTM, Embedding, Dropout, Activation, MaxPooling1D #Layers for build model

In [4]:
max_length=300
nb_max_words=100000
embedding_dim=10

In [5]:
fake_news = '/content/gdrive/MyDrive/Fake_News/Fake.csv'
true_news = '/content/gdrive/MyDrive/Fake_News/True.csv'

In [6]:
df_fake_news = pd.read_csv(fake_news)
df_true_news = pd.read_csv(true_news)

In [7]:
cList = pkl.load(open('/content/gdrive/MyDrive/Fake_News/cword_dict.pkl','rb'))

In [8]:
c_re = re.compile('(%s)' % '|'.join(cList.keys()))

In [9]:
def expandContractions(text, c_re=c_re):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text)

In [10]:
def clean_news(newses):
    cleaned_news = []
    for news in newses:
        news = str(news)
        if re.match("(\w+:\/\/\S+)", news) == None and len(news) > 5:
            news = ' '.join(re.sub("(@[A-Za-z0-9]+)|(\#[A-Za-z0-9]+)|(<Emoji:.*>)|(pic\.twitter\.com\/.*)", " ", news).split())
            news = ftfy.fix_text(news)
            news = expandContractions(news)
            news = ' '.join(re.sub("([^0-9A-Za-z \t])", " ", news).split())
            stop_words = stopwords.words('english')
            word_tokens = nltk.word_tokenize(news) 
            filtered_sentence = [w for w in word_tokens if not w in stop_words]
            news = ' '.join(filtered_sentence)
            news = PorterStemmer().stem(news)
            cleaned_news.append(news)
    return cleaned_news

In [11]:
arr_fake_news = [x for x in df_fake_news['text']]
arr_true_news = [x for x in df_true_news['text']]

In [12]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [13]:
cleaned_fake_news = clean_news(arr_fake_news)
cleaned_true_news = clean_news(arr_true_news)

AttributeError: ignored

In [14]:
from keras.preprocessing.text import one_hot

In [ ]:
files = open('/content/gdrive/MyDrive/Fake_News/tokenizer.pkl','wb')
pkl.dump(tokenizer,files)

In [15]:
# sequences_fake = tokenizer.texts_to_sequences(cleaned_fake_news)
# sequences_true = tokenizer.texts_to_sequences(cleaned_true_news)

sequences_fake=[one_hot(words,nb_max_words) for words in cleaned_fake_news]
sequences_true=[one_hot(words,nb_max_words) for words in cleaned_true_news]

In [ ]:
print(sequences_fake)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))

Found 104287 unique tokens


In [16]:
data_f = pad_sequences(sequences_fake, padding='pre', maxlen=max_length)
data_t = pad_sequences(sequences_true,padding='pre', maxlen=max_length)
print('Shape of data_fake tensor:', data_f.shape)
print('Shape of data_true tensor:', data_t.shape)

Shape of data_fake tensor: (22755, 300)
Shape of data_true tensor: (21416, 300)


In [17]:
print(data_f)

[[    0     0     0 ... 61409  1086 65542]
 [    0     0     0 ... 69772 32921 64348]
 [    0     0     0 ... 87513 20237 76067]
 ...
 [95769 90907  6583 ... 59446 12954 73009]
 [    0     0     0 ... 45038 85587 60557]
 [43080 43351  5599 ... 59446 65366 73009]]


In [18]:
data = np.concatenate((data_f, data_t))

In [19]:
print(data)

[[    0     0     0 ... 61409  1086 65542]
 [    0     0     0 ... 69772 32921 64348]
 [    0     0     0 ... 87513 20237 76067]
 ...
 [    0     0     0 ... 62727 94022 43080]
 [    0     0     0 ... 10565 74745 21809]
 [    0     0     0 ...  3237  9668 78577]]


In [20]:
labels_f = np.ones(data_f.shape[0])
labels_t = np.zeros(data_t.shape[0])
print(labels_f.shape, labels_t.shape)

(22755,) (21416,)


In [21]:
labels = np.concatenate((labels_f, labels_t))

In [22]:
X_train, X_test, Y_train, Y_test = train_test_split(data, labels, test_size = 0.2, random_state=42)

In [23]:
from keras.layers import Flatten

In [24]:
model=Sequential()
model.add(Embedding(nb_max_words,embedding_dim,input_length=max_length))
model.add(Dropout(0.7))

model.add(Conv1D(filters=16,kernel_size=3,padding='valid',activation='relu'))
model.add(MaxPooling1D())
model.add(Dropout(0.7))
model.add(Conv1D(filters=32,kernel_size=3,padding='valid',activation='relu'))
model.add(MaxPooling1D())
model.add(Dropout(0.7))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.7))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 10)           1000000   
_________________________________________________________________
dropout (Dropout)            (None, 300, 10)           0         
_________________________________________________________________
conv1d (Conv1D)              (None, 298, 16)           496       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 149, 16)           0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 149, 16)           0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 147, 32)           1568      
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 73, 32)            0

In [25]:
model.fit(X_train,Y_train,validation_data=(X_test,Y_test),epochs=10,batch_size=64)

Epoch 1/10
553/553 [==============================] - 33s 31ms/step - loss: 0.6844 - accuracy: 0.5315 - val_loss: 0.7086 - val_accuracy: 0.6544
Epoch 2/10
553/553 [==============================] - 16s 29ms/step - loss: 0.3425 - accuracy: 0.8595 - val_loss: 0.5314 - val_accuracy: 0.7540
Epoch 3/10
553/553 [==============================] - 16s 29ms/step - loss: 0.1887 - accuracy: 0.9302 - val_loss: 0.4516 - val_accuracy: 0.7978
Epoch 4/10
553/553 [==============================] - 16s 29ms/step - loss: 0.1580 - accuracy: 0.9424 - val_loss: 0.4125 - val_accuracy: 0.8114
Epoch 5/10
553/553 [==============================] - 16s 29ms/step - loss: 0.1338 - accuracy: 0.9513 - val_loss: 0.2977 - val_accuracy: 0.8645
Epoch 6/10
553/553 [==============================] - 16s 29ms/step - loss: 0.1148 - accuracy: 0.9574 - val_loss: 0.2528 - val_accuracy: 0.8918
Epoch 7/10
553/553 [==============================] - 16s 29ms/step - loss: 0.1096 - accuracy: 0.9620 - val_loss: 0.1961 - val_accuracy:

In [26]:
model.save('CNN_model.h5')